# Automated Globe Prediction in CT images of the Orbits
## Axial CT images of the orbits 

In [ ]:
"""
UNIVERSITY OF ARIZONA
Author: Lavanya Umapathy
Date: 
Description: Script to evaluate/test using a saved CNN model. 
The model uses Axial CT images of the orbits to predict globe masks.
"""

In [ ]:
from matplotlib import pyplot as plt
import time, sys
from skimage import measure

In [ ]:
# Add the necessary Utilities 
utils_dir = "../Utilities"
if utils_dir not in sys.path:
    sys.path.insert(0, utils_dir)
 
import DicomUtilities as utils

In [ ]:
# Set parameters
opShape = (512,512)

modelLoadPath = './PretrainedModel/MRes_UNET2D.h5'

dcmLoadPath = './Data/SubjectFolderName/'


In [ ]:
# Load the pretrained MRes-UNET2D model
model = utils.loadSavedModel(modelLoadPath)

# load the CT DICOM series into img (Width x Height x Number of Slices)
img = utils.loadDicomSeries_sorted(dcmPath,'IM*')

# Get pixel size for volume calculations
pixdim = utils.getPixDims_Dicom(dcmPath,'IM*')

# Pre-process CT images of the orbits
img  = utils.preProcess_orbitalCT(img,opShape,WL=50,WW=200)

In [ ]:
# Predict masks for globes using MRes-UNET2D model

start_pred = time.time()
predImg = utils.predictGlobes(model, img)
end_pred = time.time()

print("Time Elapsed prediction in seconds: ",round(end_pred - start_pred,4))

In [ ]:
# Separate the left and the right globes

PredVol_L,PredVol_R = utils.findGlobes(predImg)

# Print the total globe volume (in mL) (left + right)
print('Predicted volume :',round(utils.computeVolumeStats(predImg,pixdim),4), 'mL')

# Print the Inter Globe Volume Difference, IGVD = V_L - V_R
print('Inter Globe Volume Difference :',round(utils.computeIGVD(predImg,pixdim),4) ,'mL')


In [ ]:
# Display the predicted Globe contours

img_idx = 2
  
contours_pred = measure.find_contours(predImg[:,:,idx], 0.25)

fig, ax = plt.subplots()
ax.imshow(img[:,:,idx], interpolation='nearest', cmap=plt.cm.gray)
for n, contour in enumerate(contours_pred):
    ax.plot(contour[:, 1], contour[:, 0], linewidth=1.5, color='blue')